In [1]:
import numpy as np
import pandas as pd

In [2]:
key = pd.read_excel('../data/raw data/raw data.xlsx', sheet_name = 'conversions')

def create_dict_key(dataframe):
    '''
    '''
    conversion_key = dict()
    for year in dataframe.columns[1:]:
        values = list(dataframe[year])
        categories = list(dataframe['categories'])
        temp = dict()
        for tup in tuple(zip(categories, values)):
            temp[tup[0]] = tup[1]
        
        temp['USD'] = 1
        conversion_key[year] = temp
            
    return conversion_key

conversion_key = create_dict_key(key)

In [3]:
df = pd.read_excel('../data/raw data/raw data.xlsx', sheet_name = 'raw data')

def adjustments(raw_data, conversion_key, fy_starts=13):
    '''
    Takes inputs of raw_data and conversion key and
    converts to real money terms.
    
    Inputs:
        raw_data: pandas dataframe of raw data Excel
        raw_data: pandas dataframe of raw data Excel
        
    Outputs:
        adjusted_data: pandas dataframe of adjusted data
    '''
    columns = raw_data.columns[fy_starts:]
    df[columns] = df[columns].replace('-', np.nan)
    raw_data['Region'] = raw_data['Type']
    
    for index, row in raw_data.iterrows():
        currency = row['Currency']
        for year in columns:
            year_match = int(year[3:])
            infl = conversion_key[year_match]['Inflation Adjustment']
            curr = conversion_key[year_match][currency]
            row[year] = (float(row[year])/curr)*infl
        
        if 'US' in row['Region']:
            row['Region'] = 'US'
        elif 'Total' in row['Region']:
            row['Region'] = 'WW'
        else:
            row['Region'] = 'ExUS'
            
        raw_data.loc[index] = row
        
    clean = raw_data 
    
    return clean

clean = adjustments(df, conversion_key)

def summary(clean_data, fy_starts=13):
    '''
    Get from array 2 to array 3
    '''
    # First simulate the dataframr
    data = []
    names = list(clean_data['Proper Name'].unique())
    for name in names:
        data.append([name, 'US'])
        data.append([name, 'WW'])
        data.append([name, 'ExUS'])
    
    data = pd.DataFrame(data, columns = ['Proper Name', 'Region']) 
    
    # Add with original data to create empty rows
    years = clean_data.columns[fy_starts:]
    clean = clean_data.groupby(['Proper Name', 'Region'], as_index=False)[years].sum()
    
    clean = clean.merge(data, how='right', on=['Proper Name', 'Region'])
    
    # Place holder for summary helper
    
    return clean.fillna(0)

summary(clean)

,Proper Name,Region,FY-2007,FY-2008,FY-2009,FY-2010,FY-2011,FY-2012,FY-2013,FY-2014,...,FY-2029,FY-2030,FY-2031,FY-2032,FY-2033,FY-2034,FY-2035,FY-2036,FY-2037,FY-2038
0,Actemra,US,0.0,0.0,0.0,75.879615,208.489888,331.554468,431.294194,556.352391,...,335.725889,315.654785,335.012030,343.125495,362.802547,358.359781,363.288261,0.000000,0.000000,0.000000
1,Actemra,WW,0.0,0.0,0.0,519.382885,913.806742,1158.377021,1424.369677,1677.279130,...,1026.982709,950.614498,983.526357,1026.718156,1067.725867,1061.260590,1244.462648,0.000000,0.000000,0.000000
2,Actemra,ExUS,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Adempas,US,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,388.825103,334.938968,300.437226,296.269494,296.007724,88.168056,61.717639,0.000000,0.000000,0.000000
4,Adempas,WW,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,84.362632,...,238.420843,234.891855,204.027394,86.399891,71.240784,59.505770,26.347257,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,Yervoy,WW,0.0,0.0,0.0,0.000000,473.760000,912.999200,1226.304000,1648.995600,...,1052.208894,859.211704,529.341355,451.136010,362.668629,615.523927,600.701670,0.000000,0.000000,0.000000
281,Yervoy,ExUS,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
282,Test drug,US,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,3874.819175,3771.675564,3423.039179,2857.723248,1408.344995,2232.767493,2009.490743,1786.213994,1562.937245,1339.660496
283,Test drug,WW,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
